In [3]:
import pandas as pd
from tqdm import tqdm
from binance.client import Client

In [5]:
client = Client()


In [6]:
info = client.get_exchange_info()

In [7]:
info

{'timezone': 'UTC',
 'serverTime': 1678867334015,
 'rateLimits': [{'rateLimitType': 'REQUEST_WEIGHT',
   'interval': 'MINUTE',
   'intervalNum': 1,
   'limit': 1200},
  {'rateLimitType': 'ORDERS',
   'interval': 'SECOND',
   'intervalNum': 10,
   'limit': 50},
  {'rateLimitType': 'ORDERS',
   'interval': 'DAY',
   'intervalNum': 1,
   'limit': 160000},
  {'rateLimitType': 'RAW_REQUESTS',
   'interval': 'MINUTE',
   'intervalNum': 5,
   'limit': 6100}],
 'exchangeFilters': [],
 'symbols': [{'symbol': 'ETHBTC',
   'status': 'TRADING',
   'baseAsset': 'ETH',
   'baseAssetPrecision': 8,
   'quoteAsset': 'BTC',
   'quotePrecision': 8,
   'quoteAssetPrecision': 8,
   'baseCommissionPrecision': 8,
   'quoteCommissionPrecision': 8,
   'orderTypes': ['LIMIT',
    'LIMIT_MAKER',
    'MARKET',
    'STOP_LOSS_LIMIT',
    'TAKE_PROFIT_LIMIT'],
   'icebergAllowed': True,
   'ocoAllowed': True,
   'quoteOrderQtyMarketAllowed': True,
   'allowTrailingStop': True,
   'cancelReplaceAllowed': True,
   'i

In [10]:
symbols = [x['symbol'] for x in info['symbols']]

In [13]:
exclude = ['UP','DOWN', 'BEAR', 'BULL'] #exclude leveraged tokens

In [14]:
non_lev = [symbol for symbol in symbols if all(excludes not in symbol for excludes in exclude)]

In [17]:
relevant = [symbol for symbol in non_lev if symbol.endswith('USDT')]

In [19]:
len(relevant)

379

In [20]:
klines = {}

In [21]:
for symbol in tqdm(relevant):
    klines[symbol] = client.get_historical_klines(symbol, '1m', '1 hour ago UTC')

100%|██████████| 379/379 [00:42<00:00,  8.84it/s]


In [28]:
(pd.DataFrame(klines['BTCUSDT'])[4].astype(float).pct_change() +1).prod() -1

0.0032924753740357637

In [29]:
returns, symbols =[], []

for symbol in relevant:
    if len(klines[symbol]) > 0:
        cumret = (pd.DataFrame(klines[symbol])[4].astype(float).pct_change() +1).prod() -1
        returns.append(cumret)
        symbols.append(symbol)

In [30]:
retdf = pd.DataFrame(returns, index=symbols, columns=['ret'])

In [33]:
retdf.ret.nlargest(10)

MINAUSDT     0.059595
DUSKUSDT     0.057978
VITEUSDT     0.026007
FIROUSDT     0.025641
PNTUSDT      0.023538
AGIXUSDT     0.023035
WANUSDT      0.022614
FIDAUSDT     0.022508
MASKUSDT     0.022334
VOXELUSDT    0.022121
Name: ret, dtype: float64